# Exercise 1
 <p><div class="lev1"><a href="#Task-A.-Another-LEGO-brick-in-the-wall"><span class="toc-item-num">Task A.&nbsp;&nbsp;</span>Another LEGO brick in the wall</a></div>
 <p><div class="lev1"><a href="#Task-B.-Drop-the-Bike"><span class="toc-item-num">Task B.&nbsp;&nbsp;</span>Drop the Bike</a></div>

In [312]:
# Add your imports here
# Importing pandas and numpy
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [3]:
DATA_FOLDER = 'data'

## Task A. Another LEGO brick in the wall

LEGO is a popular brand of toy building bricks. They are often sold in sets in order to build a specific object. Each set contains a number of parts in different shapes, sizes and colors. This database contains information on which parts are included in different LEGO sets. It was originally compiled to help people who owned some LEGO sets already figure out what other sets they could build with the pieces they had.

This dataset contains the official LEGO colors, parts, inventories (i.e., sets of LEGO parts which assembled create an object in the LEGO world) and sets (i.e., sets of LEGO inventories which assembled create a LEGO ecosystem). The schema of the dataset can be shown in the following UML diagram: 

![lego-schema](lego-schema.png)

In this task you have to apply the following Data Wrangling pipeline:
1. Load your data into `Pandas`
* Explore it and clean its dirty parts
* Use it to answer a set of queries

Each of these subtasks are described in detail below.

### A1. Loading phase
Load all the csv files into different `DataFrames`. Use meaningful names for your `DataFrames` (e.g., the respective filenames).

*Hint: You can load files without first unzipping them (for `Pandas` version >= 0.18.1).*

In [3]:
LEGO_DATA_FOLDER = DATA_FOLDER + '/lego'

In [4]:
# Write your code here
colors = pd.read_csv(LEGO_DATA_FOLDER + "/colors.csv.zip")
inventories = pd.read_csv(LEGO_DATA_FOLDER + "/inventories.csv.zip")
inventory_parts = pd.read_csv(LEGO_DATA_FOLDER + "/inventory_parts.csv.zip")
inventory_sets = pd.read_csv(LEGO_DATA_FOLDER + "/inventory_sets.csv.zip")
part_categories = pd.read_csv(LEGO_DATA_FOLDER + "/part_categories.csv.zip")
parts = pd.read_csv(LEGO_DATA_FOLDER + "/parts.csv.zip")
sets = pd.read_csv(LEGO_DATA_FOLDER + "/sets.csv.zip")
themes = pd.read_csv(LEGO_DATA_FOLDER + "/themes.csv.zip")

### A2. Cleaning phase
Explore the following columns from your dataset:

1. sets: year
* inventory_parts: quantity

What is the time range of the sets? 
What is the average quantity of the inventory parts? 
Do you see any inconsistencies? 
Provide code that detects and cleans such inconsistencies and validates the coherence of your dataset. 

In [7]:
def cleanCheck(data):
    # Check whether any set has an undefined year
    is_nan = data.isna().sum() 

    # Take out the data that is incorrect
    is_bad = data.apply(lambda x: len(x) != 4).sum()
    
    print(is_nan)
    print(is_bad)
    
    

In [5]:
# Write your code here

# Cleaning sets.year 

# Count number of inconsistencies in the set
sets_bad = sets[sets['year'].apply(lambda x: len(x) != 4)]
print(sets_bad.year.count())

# Make all negative years positive
sets['year'] = sets['year'].apply(lambda x: x[1:] if x[0] == "-" else x)

# Delete last number in years with 5 digits
sets['year'] = sets['year'].apply(lambda x: x[:-1] if len(x) == 5 else x)

# Change 70s to nan
sets['year'] = sets['year'].apply(lambda x: np.nan if len(x) == 3 else x)

clean_sets = sets.dropna(axis = 0, subset=['year'])
cleanCheck(clean_sets.year)




3085


NameError: name 'cleanCheck' is not defined

In [6]:
clean_sets['year'].replace(pd.to_datetime(clean_sets.year).dt.year)


1        1978
2        1987
3        1979
4        1979
5        1979
6        1979
7        1978
8        1976
9        1976
10       1976
11       1965
12       1985
13       1976
14       1965
15       1968
16       1968
17       1968
18       1999
19       1968
20       1968
21       1968
22       1968
23       1967
24       1969
25       2001
26       2001
27       2001
28       2001
29       2001
30       2001
         ... 
11643    2014
11644    1991
11645    2012
11646    2012
11647    2013
11648    1995
11649    2003
11650    2000
11651    2000
11652    1995
11653    1998
11654    2000
11655    1985
11656    2001
11657    2000
11658    2000
11659    1997
11660    2006
11661    1997
11662    2000
11663    2003
11664    1959
11665    2006
11666    2009
11667    2013
11668    2012
11669    2015
11670    2010
11671    2013
11672    1996
Name: year, Length: 11559, dtype: object

In [7]:
# Calculate time range
start_year = clean_sets.year.min()
end_year = clean_sets.year.max()
time_range = int(clean_sets.year.max()) - int(clean_sets.year.min())
print('The time range is ' + str(time_range) + " years, between " + str(start_year) + " and " + str(end_year) + ".")

The time range is 67 years, between 1950 and 2017.


In [8]:
# Cleaning inventory_parts.quantity

# Check whether any i_p has an undefined quntity
print(inventory_parts.quantity.isna().sum())

# Replace all infinite values with 0 to simplify counting them
inventory_parts.quantity = inventory_parts.quantity.replace([np.inf, -np.inf], 0.0)

# Count amount of values equaling zero
quantity_zero = (inventory_parts['quantity'] < 1).sum()
print(quantity_zero)
print(inventory_parts.quantity.sum())

# Calculate % of values equaling zero
print("% of zero values: " + str(quantity_zero/inventory_parts.quantity.sum()))

print(inventory_parts.quantity.mean())

0
161796
1605586.0
% of zero values: 0.10077068434826911
2.7670542575540584


__\* Briefly explain your approach here \*__

#### Cleaning sets by the column year
1. First check whether all years are defined
2. Print years to see examples of inconsistencies in the data. Examples of inconsistencies are negative years, years with five digits and years written as decade+"s"(70s, 80s, etc).
3. Cleaning:
    1. First, all negative years are turned positive.
    2. Secondly, all years with 5 digits are turned into 4 digit numbers. We assume that the last digit has been repeated and therefore remove it. For example, 20011 is turned into 2001.
    3. Lastly we omit all rows where the year is written as decade+"s", as there is no way of knowing in which year of the decade the set is from.
    
#### Cleaning inventory_parts by the column quantity
1. First check whether quantity is defined for each row
2. Print quantity to see examples of inconsistencies in the data. An example of an inconsistency is that some of the quantities are written as "-inf".
3. Cleaning:
    1. Set all quantities with the value "-inf" to 0.0 instead, as 0.0 is not used in the rest of the array. This way the data will remain but functions such as count, sum etc. can be applied to the column.


### A3. Querying phase
Answer the following queries using the functionality of `Pandas`:

1. List the ids of the inventories that belong to sets that contain cars. (*Hint: Find a smart way to distinguish which sets contain cars based on the sets' name*).
* Plot the distribution of part categories as a (horizontal) bar chart. Restrict yourself to the 20 largest part categories (in terms of the number of parts belonging to the category).
* Find the dominant color of each set. Then, plot using a (horizontal) bar chart, the number of sets per dominant color. Color each bar with the respective color that it represents.
* Create a scatter plot of the *luminance*\* of the sets vs their publishing year. What do you observe for the years 1980-1981? How do you interpret what you see?

\*The luminance of a color is a [measure of brightness](https://en.wikipedia.org/wiki/Luminance) which, given its RGB representation, can be computed as follows:

$luminance = \sqrt{0.299*R^2 + 0.587*G^2 + 0.114*B^2}$

In [9]:
# Write your code here

# Get sets with "car\s" in the name
set_with_cars = sets[sets.name.str.contains(r'^(?:.*\s)?cars?(?:\s.*)?$', flags=re.IGNORECASE)]

inventory_cars = pd.merge(set_with_cars, inventory_sets, left_on='id', right_on='set_id')
print(inventory_cars)

         id                                               name  year  \
0   10002-1                                  Railroad Club Car  2001   
1   10022-1  Santa Fe Cars - Set II (dining, observation, o...  2002   
2   10022-1  Santa Fe Cars - Set II (dining, observation, o...  2002   
3   10025-1        Santa Fe Cars - Set I (mail or baggage car)  2002   
4   10025-1        Santa Fe Cars - Set I (mail or baggage car)  2002   
5   10158-1                               High Speed Train Car  2004   
6    1247-1                                         Patrol Car  1999   
7    1255-1                                     Shell Car Wash  1999   
8    1496-1                                          Rally Car  1987   
9    1517-1                                           Race Car  1989   
10   1518-1                                    Race Car Repair  1989   
11   1741-1                                                Car  1994   
12   1990-1                                        F1 Race Car  

__\* Briefly explain your approach for every query here \*__

## Task B. Drop the bike

*Los Angeles Metro* has been sharing publicly [anonymized *Metro Bike Share* trip data](https://bikeshare.metro.net/about/data/) under the [Open Database License (ODbL)](http://opendatacommons.org/licenses/odbl/1.0/).

In this task you will again perform data wrangling and interpretation.

### B1. Loading phase
Load the json file into a `DataFrame`.


In [389]:
BIKES_DATA_FOLDER = DATA_FOLDER + '/bikes'

In [390]:
# Write your code here
bikes = pd.read_json(BIKES_DATA_FOLDER + '/metro-bike-share-trip-data.json.zip')
bikes.head(10)

,Bike ID,Duration,End Time,Ending Station ID,Ending Station Latitude,Ending Station Longitude,Passholder Type,Plan Duration,Start Time,Starting Station ID,Starting Station Latitude,Starting Station Longitude,Trip ID,Trip Route Category
0,6281.0,180,2016-07-07T04:20:00,3014.0,34.056610,-118.23721,Monthly Pass,30.0,2016-07-07T04:17:00,3014.0,34.056610,-118.23721,1912818,Round Trip
1,6281.0,1980,2016-07-07T06:33:00,3014.0,34.056610,-118.23721,Monthly Pass,30.0,2016-07-07T06:00:00,3014.0,34.056610,-118.23721,1919661,Round Trip
2,5861.0,300,2016-07-07T10:37:00,3016.0,34.052898,-118.24156,Flex Pass,365.0,2016-07-07T10:32:00,3016.0,34.052898,-118.24156,1933383,Round Trip
3,5861.0,10860,2016-07-07T13:38:00,3016.0,34.052898,-118.24156,Flex Pass,365.0,2016-07-07T10:37:00,3016.0,34.052898,-118.24156,1944197,Round Trip
4,6674.0,420,2016-07-07T12:58:00,3032.0,34.049889,-118.25588,Walk-up,0.0,2016-07-07T12:51:00,3032.0,34.049889,-118.25588,1940317,Round Trip
5,6717.0,780,2016-07-07T13:04:00,3054.0,34.039219,-118.23649,Monthly Pass,30.0,2016-07-07T12:51:00,3021.0,34.045609,-118.23703,1944075,One Way
6,5721.0,600,2016-07-07T13:04:00,3014.0,34.056610,-118.23721,Monthly Pass,30.0,2016-07-07T12:54:00,3022.0,34.046070,-118.23309,1944073,One Way
7,5957.0,600,2016-07-07T13:09:00,3005.0,34.048550,-118.25905,Flex Pass,365.0,2016-07-07T12:59:00,3076.0,34.040600,-118.25384,1944067,One Way
8,6137.0,2880,2016-07-07T13:49:00,3031.0,34.044701,-118.25244,Flex Pass,365.0,2016-07-07T13:01:00,3031.0,34.044701,-118.25244,1944062,Round Trip
9,6351.0,960,2016-07-07T13:17:00,3078.0,34.064281,-118.23894,Monthly Pass,30.0,2016-07-07T13:01:00,3031.0,34.044701,-118.25244,1944063,One Way


### B2. Cleaning phase
Describe the type and the value range of each attribute. Indicate and transform the attributes that are `Categorical`. Are there redundant columns in the dataset (i.e., are there columns whose value depends only on the value of another column)? What are the possible pitfalls of having such columns? Reduce *data redundancy* by extracting such columns to separate `DataFrames`. Which of the two formats (the initial one or the one with reduced data redundancy) is more susceptible to inconsistencies? At the end print for each `Dataframe` the *type of each column* and it's *shape*.

In [391]:
# Write your code here

# Check with shape to get amount of columns


# Check min and max to get value range
bikes_max_min = bikes.describe(include=[np.number]).loc[['min','max']]
value_range = bikes_max_min.loc['max'] - bikes_max_min.loc['min']

# Print for all numerical values
[print(
    '{0:1}:\n Type: {1:1},\n Max: {2:.6f},\n Min: {3:.6f},\n Value range: {4:0.6f}'.
    format(
        i,
        str(bikes.dtypes.loc[i]),
        bikes_max_min.loc['max', i],
        bikes_max_min.loc['min', i],
        value_range[i]
    )
) for i in bikes_max_min.columns]

bikes_date_max = bikes.get(['End Time', 'Start Time']).max()
bikes_date_min = bikes.get(['End Time', 'Start Time']).min()
bikes_value_range = bikes_date_max - bikes_date_min



Bike ID:
 Type: float64,
 Max: 6728.000000,
 Min: 1349.000000,
 Value range: 5379.000000
Duration:
 Type: int64,
 Max: 86400.000000,
 Min: 60.000000,
 Value range: 86340.000000
Ending Station ID:
 Type: float64,
 Max: 4108.000000,
 Min: 3000.000000,
 Value range: 1108.000000
Ending Station Latitude:
 Type: float64,
 Max: 34.064281,
 Min: 0.000000,
 Value range: 34.064281
Ending Station Longitude:
 Type: float64,
 Max: 0.000000,
 Min: -118.472832,
 Value range: 118.472832
Plan Duration:
 Type: float64,
 Max: 365.000000,
 Min: 0.000000,
 Value range: 365.000000
Starting Station ID:
 Type: float64,
 Max: 4108.000000,
 Min: 3000.000000,
 Value range: 1108.000000
Starting Station Latitude:
 Type: float64,
 Max: 34.064281,
 Min: 0.000000,
 Value range: 34.064281
Starting Station Longitude:
 Type: float64,
 Max: 0.000000,
 Min: -118.472832,
 Value range: 118.472832
Trip ID:
 Type: int64,
 Max: 23794218.000000,
 Min: 1912818.000000,
 Value range: 21881400.000000


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [392]:
# The columns 'Trip Route Category' and 'Passholder Type' are objects of string characters
# which can be mapped to categories
bikes['Trip Route Category'] = bikes['Trip Route Category'].astype('category')
bikes['Passholder Type'] = bikes['Passholder Type'].astype('category')
print(bikes.max())

# For 'Passholder Type', we see that 'Plan Duration' is NaN for the category 'Staff Annual'.
# Since the name is annual, we change NaN to 365
index_staff_annual = bikes[bikes['Passholder Type'].apply(lambda x: x == 'Staff Annual')].index
bikes.loc[index_staff_annual, 'Plan Duration'] = 365

# Check if and in that case which categories have remaining NaN values
bikes[bikes['Plan Duration'].isnull()]['Passholder Type'].value_counts()

# Get index for remaining nan values in category 'Monthly Pass'
index_remaining_nan = bikes[bikes['Passholder Type'].apply(lambda x: x == 'Monthly Pass') & 
                            bikes['Plan Duration'].isnull()].index

# Only 'Monthly Pass' has NaN values, change these to 30
bikes.loc[index_remaining_nan, 'Plan Duration'] = 30

# Drop all rows where latitude (if latitude is missing, so is longitude) or bike id is missing 
bikes = bikes.dropna(subset=['Ending Station Latitude', 'Starting Station Latitude', 'Bike ID'])

# The columns which depends on other columns are 'Duration', 'Plan Duration' and 'Trip Route Category'
# sista är för att den kan beräknas med hjälp av att jämföra start och end station id
duration = bikes[['Duration']]
plan_duration = bikes[['Plan Duration']]
t_r_c = bikes[['Trip Route Category']]

# bikes = bikes.drop(['Duration', 'Plan Duration', 'Trip Route Category'], axis=1)


Bike ID                                      6728
Duration                                    86400
End Time                      2017-04-02T10:32:00
Ending Station ID                            4108
Ending Station Latitude                   34.0643
Ending Station Longitude                        0
Passholder Type                           Walk-up
Plan Duration                                 365
Start Time                    2017-03-31T23:45:00
Starting Station ID                          4108
Starting Station Latitude                 34.0643
Starting Station Longitude                      0
Trip ID                                  23794218
Trip Route Category                    Round Trip
dtype: object


In [393]:
# We also can match the station to their latitude and longitude, which can be saved in a seperate Dataframe
# A Station ID can have two different longitudes. We round these values which make them correspond and then drop
# the duplicates. The values are sorted and the table is reindexed.
stationID = pd.concat(
    [bikes[['Ending Station ID', 'Ending Station Latitude', 'Ending Station Longitude']
          ].rename(index=str, columns={"Ending Station ID": "Station ID",
                            "Ending Station Latitude": "Latitude",
                            "Ending Station Longitude": "Longitude" }),
     bikes[['Starting Station ID', 'Starting Station Latitude', 'Starting Station Longitude']
          ].rename(index=str, columns={"Starting Station ID": "Station ID",
                            "Starting Station Latitude": "Latitude",
                            "Starting Station Longitude": "Longitude" })],
    sort=False).round(5).drop_duplicates().sort_values(by=['Station ID']).reset_index(drop=True)

In [394]:
# Drop latitude and longitude for end and start in bikes
bikes = bikes.drop(['Ending Station Latitude', 'Ending Station Longitude',
                    'Starting Station Latitude', 'Starting Station Longitude'], axis=1)

In [395]:
# Print all frames

# bikes
print('The datatype of each column in bikes:')
print(bikes.dtypes)
print("\nThe shape of bikes is", bikes.shape)

# duration
print('\nThe datatype of each column in duration:')
print(duration.dtypes)
print("\nThe shape of duration is", duration.shape)

# plan_duration
print('\nThe datatype of each column in plan_duration:')
print(plan_duration.dtypes)
print("\nThe shape of plan_duration is", plan_duration.shape)

# t_r_c
print('\nThe datatype of each column in t_r_c:\n')
print(t_r_c.dtypes)
print("\nThe shape of t_r_c is", t_r_c.shape)

# stationID
print('\nThe datatype of each column in stationID:\n')
print(stationID.dtypes)
print("\nThe shape of stationID is", stationID.shape)

The datatype of each column in bikes:
Bike ID                 float64
Duration                  int64
End Time                 object
Ending Station ID       float64
Passholder Type        category
Plan Duration           float64
Start Time               object
Starting Station ID     float64
Trip ID                   int64
Trip Route Category    category
dtype: object

The shape of bikes is (131327, 10)

The datatype of each column in duration:
Duration    int64
dtype: object

The shape of duration is (131327, 1)

The datatype of each column in plan_duration:
Plan Duration    float64
dtype: object

The shape of plan_duration is (131327, 1)

The datatype of each column in t_r_c:

Trip Route Category    category
dtype: object

The shape of t_r_c is (131327, 1)

The datatype of each column in stationID:

Station ID    float64
Latitude      float64
Longitude     float64
dtype: object

The shape of stationID is (67, 3)


__\* Briefly explain your approach here \*__

#### Check type and value range of each attribute
1. First check type and value range of numerical values
2. Then do it for object types

#### Categorical
1. First check which categories are based on string characters
2. Then transform these to the Category type

#### Data redundancy
1. Redundant columns are 'Duration', 'Plan Duration' and 'Trip Route Category', since all these can be calculated from other columns.
2. We extract station IDs, latitude and longitude to create a new dataframe where the IDs are matched with the longitudes
3. Pitfalls of having redundant columns are that they take up memory space, if data is compromised in that column the entire is dirty without having to be and it is harder for a human to handle and oversee a big dataset.
4. The initial one is more susceptible to inconsistencies. For example, if one value of Duration has been accidentally set to an incorrect value in the first dataset it is not sure the user will notice this. If Duration is calculated based on End Time and Start Time, it will give the correct Duration between these times for each row. However this does not help if End Time or Start Time has been compromised. Another issue that might create inconsistencies for the one with reduced data redundancy is if operations are made on the extracted datasets that change their shape/size, indices etc. so the datasets no longer corresponds to the first dataset. Therefore you have to make sure the data still matches.

### B3. Querying phase
Answer the following queries using the functionality of `Pandas`.

1. Plot the *distribution* of the number of outgoing trips from each station in a histogram with 20 bins (Hint: each bin describes a range of counts, not stations).
* Plot histograms for the *duration* and *trip starting hour in the day* attributes. For both the *duration*  and the *trip starting hour* use *discrete 1-hour intervals*. What do you observe in each plot? What are some popular values in the *duration* plot? Explain the local maxima and the trends you observe on the *trip starting hour* plot based on human behavior.
* For each *trip route category*, calculate the proportion of trips by *passholder type* and present your results in *a stacked bar chart with normalized height*.
* Considering only trips that begin in the morning hours (before noon), plot in *a single bar chart* the proportion of trips by *passholder type* and *trip route category*. Explain any outliers you observe.
* Separate the hours of the day into two intervals that have (approximately) the same number of bikes leaving the stations. For each of the two intervals calculate the proportion of trips by *passholder type* and *trip route category*. Present your results in a `DataFrame` which has a unique, non-composite index. Does the proportion of trips depend on whether it is the first or second hour interval? Would the company have any significant benefit by creating a more complex paying scheme where monthly pass users would pay less in the first interval and (equally) more on the second one? Assume that the number of trips per interval will not change if the scheme changes.

In [13]:
# Write your code here

__\* Briefly explain your approach for every query here \*__